In [286]:
import os
import tensorflow as tf
from tensorflow.contrib.learn.python.learn.datasets import mnist
import numpy as np
import pandas as pd

In [300]:
save_dir = "save_dir"
save_dir_TFRecord_01 = "save_dir_TFRecord_01"

In [301]:
#save_dir 에 데이터 내려받기
data_sets = mnist.read_data_sets(save_dir, 
                                 dtype=tf.uint8,
                                reshape=False,
                                validation_size=1000)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting save_mnist_test/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting save_mnist_test/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting save_mnist_test/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting save_mnist_test/t10k-labels-idx1-ubyte.gz


In [289]:
print('{}\n[0]{}\n[1]{}\n[2]{}'.format(type(data_sets),data_sets[0],data_sets[1],data_sets[2]))
print('[0].image.shape:{}\n    labels.shape:{}'.format(data_sets[0].images.shape,data_sets[0].labels.shape))
print('[1].image.shape:{}\n    labels.shape:{}'.format(data_sets[1].images.shape,data_sets[1].labels.shape))
print('[2].image.shape:{}\n    labels.shape:{}'.format(data_sets[2].images.shape,data_sets[2].labels.shape))


<class 'tensorflow.contrib.learn.python.learn.datasets.base.Datasets'>
[0]<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x7ff31de2cf28>
[1]<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x7ff31e003e48>
[2]<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x7ff31de2cda0>
[0].image.shape:(59000, 28, 28, 1)
    labels.shape:(59000,)
[1].image.shape:(1000, 28, 28, 1)
    labels.shape:(1000,)
[2].image.shape:(10000, 28, 28, 1)
    labels.shape:(10000,)


In [299]:
print_onoff = 0
data_splits = ["train","test","validation"]
for d in range(len(data_splits)):
    print('*'*100)
    print("saving "+data_splits[d])
    data_set = data_sets[d]
    filename = os.path.join(save_dir_TFRecord_01, data_splits[d] + '.tfrecords')
    print('Loop({}) filename : {}'.format(d, filename))
    writer = tf.python_io.TFRecordWriter(filename)
    for index in range(data_set.images.shape[0]):
        image = data_set.images[index].tostring()
        example = tf.train.Example(features=tf.train.Features(
            feature={
                'height': tf.train.Feature(
                    int64_list=tf.train.Int64List(value=[data_set.images.shape[1]])),     # 실데이터가 아닌, image_row의 height Meta값을 저장한다.
                'width': tf.train.Feature(
                    int64_list=tf.train.Int64List(value=[data_set.images.shape[2]])),     # 실데이터가 아닌, image_row의 width Meta값을 저장한다.
                'depth': tf.train.Feature(
                    int64_list=tf.train.Int64List(value=[data_set.images.shape[3]])),     # 실데이터가 아닌, image_row의 depth Meta값을 저장한다.
                'label': tf.train.Feature(
                    int64_list=tf.train.Int64List(value=[int(data_set.labels[index])])),  # label의 실제값을 저장한다.
                'image_raw': tf.train.Feature(
                    bytes_list=tf.train.BytesList(value=[image]))}))                      # image의 실제값을 저장하다.
        print_onoff += 1
        if print_onoff == 1:
            #print("     >> type(example.features.feature) : {}\n     >> example.features.feature['height'] : {}".format(type(example.features.feature),example.features.feature["height"]))
            print("     >> type(example.features.feature['image_row']) : {}".format(type(example.features.feature['image_raw'])))
            print("     >> type(image) : {}   |   len(image) : {}".format(type(image),len(image)))
        writer.write(example.SerializeToString())
    writer.close()
    print('     >> Loop({}) Complete!'.format(d))
print('*'*100)
print('complete!')

****************************************************************************************************
saving train
Loop(0) filename : save_dir_TFRecord_01/train.tfrecords
     >> type(example.features.feature['image_row']) : <class 'tensorflow.core.example.feature_pb2.Feature'>
     >> type(image) : <class 'bytes'>   |   len(image) : 784
     >> Loop(0) Complete!
****************************************************************************************************
saving test
Loop(1) filename : save_dir_TFRecord_01/test.tfrecords
     >> Loop(1) Complete!
****************************************************************************************************
saving validation
Loop(2) filename : save_dir_TFRecord_01/validation.tfrecords
     >> Loop(2) Complete!
****************************************************************************************************
complete!


# tostring에 대해서 알아보자

In [98]:
x1 = np.array([1,2], np.uint8)
x2 = np.array([1,2], np.uint16)
x3 = np.array([1,2], np.uint32)
x4 = np.array([1,2], np.uint64)
print('np.uint8 => tostring result : {}'.format(x1.tostring()))
print('np.uint16 => tostring result : {}'.format(x2.tostring()))
print('np.uint32 => tostring result : {}'.format(x3.tostring()))
print('np.uint64 => tostring result : {}'.format(x4.tostring()))

np.uint8 => tostring result : b'\x01\x02'
np.uint16 => tostring result : b'\x01\x00\x02\x00'
np.uint32 => tostring result : b'\x01\x00\x00\x00\x02\x00\x00\x00'
np.uint64 => tostring result : b'\x01\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00'


In [112]:
x1 = np.array([1,2], np.uint8)
x2 = np.array([513], np.uint16)
print('np.uint8  => tostring result : {}'.format(x1.tostring()))
print('np.uint16 => tostring result : {}'.format(x2.tostring()))

#for i in range(1000):
#    print(i, "|", np.array([i], np.uint16).tostring())

np.uint8  => tostring result : b'\x01\x02'
np.uint16 => tostring result : b'\x01\x02'


# DataFrame으로 넣고 데이터 직관적으로 확인하기

In [179]:
data_set.images[0]
data_set_df_pre = data_set.images[1].reshape(28,28)
data_set_df = pd.DataFrame(data_set_df_pre)
data_set_df

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,169,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,169,253,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,52,250,253,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,77,251,210,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,31,18,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [291]:
# 이진 형태로 저장된 파일에서 1건씩 불러오는 과정
filename = os.path.join(save_dir_TFRecord_01, 'validation.tfrecords')
record_iterator = tf.python_io.tf_record_iterator(filename)
seralized_img_example = next(record_iterator)

In [284]:
# # 다음 Record 불러옴.
# seralized_img_example = next(record_iterator)

In [285]:
example = tf.train.Example()
example.ParseFromString(seralized_img_example)
image = example.features.feature['image_raw'].bytes_list.value
label = example.features.feature['label'].int64_list.value[0]
width = example.features.feature['width'].int64_list.value[0]
height = example.features.feature['height'].int64_list.value[0]

In [271]:
img_flat = np.fromstring(image[0], dtype=np.uint8)
img_reshaped = img_flat.reshape((28, 28, -1))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  """Entry point for launching an IPython kernel.


In [292]:
print('type : {}\nshape : {}'.format(type(img_flat), img_flat.shape))
print('{} '.format(img_flat, img_reshaped))

type : <class 'numpy.ndarray'>
shape : (784,)
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0  84 185 159 151  60  36   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0 222 254 254 254
 254 